In [ ]:
# Fixing the AI-Powered E-Commerce Chatbot Code for Google Colab

# Install Required Libraries
!pip install -q langchain langchain_community transformers accelerate einops bitsandbytes chainlit sentence-transformers faiss-cpu pypdf pyngrok

# Import Dependencies
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts.prompt import PromptTemplate
import chainlit as cl
from pyngrok import ngrok
from langchain.memory import ConversationBufferMemory
from torch import cuda, bfloat16
from langchain.chains import RetrievalQA

# Set up Hugging Face Embeddings
model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cuda' if cuda.is_available() else 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Set up LLM Model (Hugging Face Auth Required)
model_id = 'meta-llama/Llama-2-7b-chat-hf'

# Ensure GPU Compatibility
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16 if torch.cuda.is_available() else torch.float32
)

# Authenticate Hugging Face (Manually Input API Key in Colab)
hf_auth = "your hugging face api"

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto' if cuda.is_available() else 'cpu',
    use_auth_token=hf_auth
)
model.eval()

# Load Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth,
)

# Setup Text Generation Pipeline
pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    temperature=0.2,
    max_new_tokens=512,
    repetition_penalty=1.1,
)

# Load and Process Datasets
loader = CSVLoader(
    file_path="/content/customers-10000.csv",
    csv_args={"delimiter": ","},
)
data = loader.load()

pdf_loader = PyPDFLoader("kb-customer-faq.pdf")
docs = pdf_loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=15)
texts = text_splitter.split_documents(docs)

# Create Embeddings and Store in FAISS
db = FAISS.from_documents(data, embeddings)
db2 = FAISS.from_documents(texts, embeddings)
db.merge_from(db2)
db.save_local("faiss_index")

print("Setup complete! The chatbot is ready.")




/usr/local/lib/python3.11/dist-packages/transformers/models/auto/configuration_auto.py:1039: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Device set to use cuda:0


Setup complete! The chatbot is ready.


# New Section